In [51]:
import simpy 
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

In [52]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random
import simpy

waiting_times = []
service_times = []

def source(env, number, lambd, mu, counter, B_distribution, shortest_job_first=False):
    """Source generates customers randomly"""
    t0 = np.random.exponential(1/lambd) # Arrival time of first customer
    yield env.timeout(t0)

    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, mu, B_distribution, shortest_job_first)
        env.process(c)

        # A distribution
        t = np.random.exponential(1/lambd)
        yield env.timeout(t)
   
def customer(env, name, counter, mu, B_distribution, shortest_job_first=False):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    # print('%7.4f %s: Here I am' % (arrive, name))
    
    # B distribution
    service_time = B_distribution() # np.random.exponential(1/mu)
    # print(service_time)

    priority = service_time if shortest_job_first else 1

    with counter.request(priority=priority) as req: 
        yield req
        wait = env.now - arrive
        waiting_times.append(wait)
        # print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        service_times.append(service_time)
        yield env.timeout(service_time)
        # print('%7.4f %s: Finished' % (env.now, name))


In [54]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 10000 # Total number of customers
LAMBD = 1/12
MU = 1/11
random.seed(RANDOM_SEED)

def mmc_sim(new_customers, lambd, mu, capacity, B_distribution, shortest_job_first=False):
    global waiting_times
    global service_times
    waiting_times = []
    service_times = []
    
    env = simpy.Environment()

    # Start processes and run
    counter = simpy.PriorityResource(env, capacity=capacity)
    env.process(source(env, new_customers, lambd, mu, counter, B_distribution, shortest_job_first))
    env.run()
    
    return waiting_times, service_times

In [55]:
def long_tail_distribution(avg_1, avg_2, chance_1):
    if np.random.rand() < chance_1:
        return np.random.exponential(scale=avg_1)
    else:
        return np.random.exponential(scale=avg_2)

In [57]:
def queueing_simulations(new_customers, lambd, mu, capacity, n_samples):
    global waiting_times
    global service_times
    dict_wt = {}
    dict_st = {}

    for i in range(n_samples):
        waiting_times = []
        service_times = []
        waiting_times, service_times = mmc_sim(new_customers, lambd, mu, capacity, lambda: np.random.exponential(1/mu))
        dict_wt[i] = waiting_times
        dict_st[i] = service_times

    return dict_wt, dict_st

result = queueing_simulations(5, 1/12, 1/11, 1, 3)
print(result[0])

{0: [0.0, 7.594698531525701, 0.24288081848006016, 0.0, 0.0], 1: [0.0, 0.0, 12.313816527662695, 27.54335208161156, 32.30528821626501], 2: [0.0, 51.53176912926014, 68.75741497751464, 50.78883081395334, 43.605594241899155]}


In [58]:
# Start out with mu = 0.1, rho = 0.9

# Variation of simulation function where you input rho and the function calculates a lambda 
def queueing_simulations2(new_customers, rho, mu, capacity, n_samples):
    global waiting_times
    global service_times

    lambd = rho * capacity * mu 
    dict_wt = {}
    dict_st = {}

    for i in range(n_samples):
        print(i)
        waiting_times = []
        service_times = []
        waiting_times, service_times = mmc_sim(new_customers, lambd, mu, capacity, lambda: np.random.exponential(1/mu))
        dict_wt[i] = waiting_times
        dict_st[i] = service_times

    return dict_wt, dict_st

In [59]:
result_n1 = queueing_simulations2(1000, 0.9, 0.1, 1, 50)
result_n2 = queueing_simulations2(1000, 0.9, 0.1, 2, 50)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [60]:
def compute_meanwt(result_dict):
    l_mean_wt = []
    for key in result_dict.keys(): 
        l_mean_wt.append(np.mean(result_dict[key]))
    mean_wt = np.mean(l_mean_wt)
    return mean_wt

def compute_varwt(result_dict):
    l_mean_wt = []
    for key in result_dict.keys(): 
        l_mean_wt.append(np.mean(result_dict[key]))
    var_wt = np.var(l_mean_wt)
    return var_wt

print(compute_meanwt(result_n1[0]))
print(compute_varwt(result_n1[0]))
print(compute_meanwt(result_n2[0]))
print(compute_varwt(result_n2[0]))

86.87550457681652
2022.7610514334158
33.911271869703384
239.42206695755303


In [63]:
def theoretical_mean_wait(labda, mu, capacity):
    rho = labda/(capacity*mu)

    p_0_inverse = 0
    for n in range(capacity):
        p_0_inverse += (capacity*rho)**n/math.factorial(n) + ((capacity*rho)**capacity/math.factorial(capacity))/(1-rho)

    p_0 = 1/p_0_inverse
    p_c = (capacity*rho)**capacity/math.factorial(capacity) * p_0
    delay_probability = p_c/(1-rho)
    E_W = delay_probability * (1/(1-rho)) * 1/(capacity*mu)

    return E_W
    # print(f'Theoretical mean waiting time = {E_W}')

# print(f'Empirical mean waiting time = {np.mean(waiting_times)}')